In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
from timm import create_model

# 데이터셋 경로
data_dir = '/home/work/team3/dataset'

# 이미지 변환 클래스
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=1.0):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + f'(mean={self.mean}, std={self.std})'

# 데이터 전처리 정의
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomRotation(20),
        transforms.RandomVerticalFlip(),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Lambda(lambda x: AddGaussianNoise(mean=0.0, std=0.1)(x))
    ]),
    'val': transformsimport torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
from timm import create_model

# 데이터셋 경로
data_dir = '/home/work/team3/dataset'

# 이미지 변환 클래스
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=1.0):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + f'(mean={self.mean}, std={self.std})'

# 데이터 전처리 정의
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomRotation(20),
        transforms.RandomVerticalFlip(),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Lambda(lambda x: AddGaussianNoise(mean=0.0, std=0.1)(x))
    ]),.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),  # 검증 데이터에도 적용
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 데이터셋 로드
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=data_transforms['train'])
valid_dataset = datasets.ImageFolder(os.path.join(data_dir, 'valid'), transform=data_transforms['val'])

# 클래스명
class_names = train_dataset.classes

# 데이터 로더
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 로드 및 수정
model = create_model('efficientnet_b1', pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, len(class_names))
)
model = model.to(device)

# 손실 함수 및 최적화
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)

# 스케줄러 및 Early Stopping 준비
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
early_stopping_counter = 0
early_stopping_limit = 10

# 모델 학습 및 검증 루프
num_epochs = 100
best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # 각 페이즈 (학습/검증)
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        # 데이터 배치 처리
        for inputs, labels in tqdm(train_loader if phase == 'train' else valid_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset if phase == 'train' else valid_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset if phase == 'train' else valid_loader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Early Stopping 체크
        if phase == 'val':
            if epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
                early_stopping_counter = 0
            else:
                early_stopping_counter += 1

            if early_stopping_counter >= early_stopping_limit:
                print("Early stopping triggered.")
                break

    if early_stopping_counter >= early_stopping_limit:
        break

# 모델 저장
import os

# 모델 저장 경로 설정
save_path = '/home/work/team3/model2.pt'

# 저장 경로의 디렉토리가 없는 경우 생성
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 모델 저장
torch.save(model.state_dict(), save_path)
print("Training complete and model saved.")


Using device: cuda
Epoch 1/100
----------


100% 149/149 [01:01<00:00,  2.44it/s]


train Loss: 5.2303 Acc: 0.0168


100% 18/18 [00:03<00:00,  5.83it/s]


val Loss: 5.0149 Acc: 0.0614
Epoch 2/100
----------


...

val Loss: 1.5251 Acc: 0.6272
Epoch 35/100
----------


100% 149/149 [01:00<00:00,  2.48it/s]


train Loss: 0.7263 Acc: 0.8214


100% 18/18 [00:03<00:00,  5.89it/s]


val Loss: 1.5518 Acc: 0.6158
Early stopping triggered.
Training complete and model saved.
